In [2]:
import sys
from collections import Counter, OrderedDict
from collections import defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

#from inverted_index_colab import *
from binary_inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update
!apt install openjdk-8-jdk-headless -qq 
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     |████████████████████████████████| 281.3 MB 32 kB/s 
     |████████████████████████████████| 198 kB 51.1 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://archive.ubuntu.com/ubuntu bionic-ba

In [4]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [5]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [6]:
spark

In [7]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [8]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].
Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
- [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [9]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [10]:
# take the 'title' and 'id' or the first 1000 rows and create an RDD from it
doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd

# take the 'anchor' and 'id' or the first 1000 rows and create an RDD from it
doc_anchor_pairs = parquetFile.limit(1000).select("anchor_text", "id").rdd

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

In [12]:
def word_to_docid(anchors, id):
  ''' 
  -----------
    title: str
      Title of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, doc_id) pairs 
      for example: [("Anarchism", 12), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(anchors.lower())]

  tuple_list = [] # creating the list
  for term in np.unique(tokens): # we only need a unique occurence of each term since the search on it will be binary
    tuple_list.append((term, id)) # adding to the list

  return tuple_list

In [13]:
def g(x):
  print(x)

In [14]:
#print(word_to_docid(doc_anchor_pairs.collect()[0][0], doc_anchor_pairs.collect()[0][1]))
#word_docid = doc_title_pairs.flatMap(lambda x: word_to_docid(x[0], x[1]))
#print(word_docid.collect())
#print(type(word_docid))
#Create an anchor list per doc id
anchors_lists = doc_anchor_pairs.flatMap(lambda tup: tup).collect()[::2]
flat_list = [item for sublist in anchors_lists for item in sublist]
anchor_docid =list(map(lambda x:  word_to_docid(x[1],x[0]),flat_list))
anchor_docid = [item for sublist in anchor_docid for item in sublist]
anchor_docid = sc.parallelize(anchor_docid).map(lambda x: x)
print(anchor_docid.collect()[:20])

[('philosophy', 23040), ('political', 23040), ('movement', 99232), ('authority', 170653), ('hierarchy', 13998), ('state', 28151), ('far-left', 18247344), ('libertarian', 3661758), ('marxism', 3661758), ('libertarian', 3225498), ('libertarian', 18048), ('socialism', 18048), ('movement', 47246185), ('socialist', 47246185), ('anti-capitalism', 44443), ('socialism', 26847), ('anarchy', 1063286), ('history', 1063286), ('prehistory', 18994022), ('realm', 5612299)]


In [15]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(list(set(unsorted_pl)))

In [17]:
anchor_postings = anchor_docid.groupByKey().mapValues(reduce_word_counts)
print(anchor_postings.collect()[:20])
print(type(anchor_postings))

[('philosophy', [1167, 4468, 5312, 5822, 9258, 12393, 23003, 23012, 23040, 23612, 24269, 24983, 30340, 37010, 39872, 46439, 80666, 151419, 159211, 166162, 166200, 171171, 175456, 180786, 192007, 209565, 219447, 223389, 307365, 309641, 309909, 310169, 357356, 382251, 461391, 730473, 803178, 806549, 806560, 1688138, 1967949, 1979016, 2110950, 2552726, 2695116, 2816803, 2890279, 2906083, 2958015, 3064846, 3144227, 3331852, 3613142, 3792917, 4870766, 5483237, 5646829, 6098864, 6880370, 6880483, 7939344, 8927416, 10649725, 11295944, 13291712, 13549878, 13692155, 13704154, 13800309, 14298096, 18718508, 21031297, 26571896, 27885874, 29016566, 31110831, 32555744, 34663672, 40341188, 41088542]), ('political', [9457, 11175, 12229, 15181, 18173, 18499, 22986, 23040, 23490, 23996, 24388, 28661, 32026, 37307, 45446, 46044, 147885, 172077, 191829, 221737, 370588, 396356, 408396, 445407, 486842, 574823, 958988, 976102, 1019726, 1205302, 1275114, 1329211, 1341522, 1621432, 1700192, 1728372, 1830882, 2

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
 
  '''
  taking a posting of the form (w, posting_list)
  and returning (bucket_id, (w, posting_list))
  '''

  def mapping_bucket_id(term, posting):
    return (token2bucket_id(term), posting)

  def write_id(id_mapped_postings):
    return InvertedIndex.write_a_posting_list(id_mapped_postings)

  locations = postings.map(lambda x: mapping_bucket_id(x[0], x)).groupByKey().map(lambda x: write_id(x))

  return locations
  # def mapping_bucket_id(posting):
  #   return (token2bucket_id(posting[0]), posting)
  
  # def write_id(id_mapped_postings):
  #   return InvertedIndex.write_a_posting_list(id_mapped_postings)

  # locations = postings.flatMap(lambda x: )
  # locations = postings.map(mapping_bucket_id).groupByKey().map(write_id)
  
  #return locations

In [ ]:
posting_locs_list = partition_postings_and_write(postings).collect()
print(posting_locs_list)

In [20]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  def to_df(posting):
    return (posting[0], len(posting[1]))

  return postings.map(to_df)

In [21]:
# global statistics
w2df = calculate_df(anchor_postings)
w2df_dict = w2df.collectAsMap()

In [ ]:
# merge the posting locations into a single dict and run more tests (5 points)
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')

In [ ]:
TUPLE_SIZE = 4       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      posting_list.append(doc_id)
    return posting_list

In [ ]:
pl = read_posting_list(inverted, 'alabama')
print(pl)

[303, 1285, 1286, 1484]
